### Convert a GFF3 downloaded from NCBI to a BED file. (Using E. coli ref genome as an example)

In [46]:
import pandas as pd

#### Read in the file using pandas. The first 3 lines of the file can be skipped because they do not contain any relevant gene/CDS information

In [47]:
df=pd.read_csv('NC_000913.2.gff3.txt',sep='\t',header=None,skiprows=3) #Skip first three lines in the file
df.drop(1,axis=1,inplace=True)  #drop column 1 
df.head()


,0,2,3,4,5,6,7,8
0,NC_000913.2,gene,190,255,.,+,.,"ID=gene-b0001;Dbxref=EcoGene:EG11277,GeneID:94..."
1,NC_000913.2,CDS,190,255,.,+,0,ID=cds-NP_414542.1;Parent=gene-b0001;Dbxref=AS...
2,NC_000913.2,gene,337,2799,.,+,.,"ID=gene-b0002;Dbxref=EcoGene:EG10998,GeneID:94..."
3,NC_000913.2,CDS,337,2799,.,+,0,ID=cds-NP_414543.1;Parent=gene-b0002;Dbxref=AS...
4,NC_000913.2,gene,2801,3733,.,+,.,"ID=gene-b0003;Dbxref=EcoGene:EG10999,GeneID:94..."


#### The file has duplicates because it lists both genes and CDS. For this file I only want to keep the genes

In [48]:
df_gene=df.loc[df[2] == 'gene'].copy()
df_gene.head()

,0,2,3,4,5,6,7,8
0,NC_000913.2,gene,190,255,.,+,.,"ID=gene-b0001;Dbxref=EcoGene:EG11277,GeneID:94..."
2,NC_000913.2,gene,337,2799,.,+,.,"ID=gene-b0002;Dbxref=EcoGene:EG10998,GeneID:94..."
4,NC_000913.2,gene,2801,3733,.,+,.,"ID=gene-b0003;Dbxref=EcoGene:EG10999,GeneID:94..."
6,NC_000913.2,gene,3734,5020,.,+,.,"ID=gene-b0004;Dbxref=EcoGene:EG11000,GeneID:94..."
8,NC_000913.2,gene,5234,5530,.,+,.,"ID=gene-b0005;Dbxref=EcoGene:EG14384,GeneID:94..."


#### The info for gene names is contained in column 8, so we want to extract the gene name for each gene interval and make a new column with gene names

In [49]:
df_gene.loc[:,('names')]=df_gene.apply(lambda x:x[8].split(';')[2].split('=')[1],axis=1).values
df_gene.head()

,0,2,3,4,5,6,7,8,names
0,NC_000913.2,gene,190,255,.,+,.,"ID=gene-b0001;Dbxref=EcoGene:EG11277,GeneID:94...",thrL
2,NC_000913.2,gene,337,2799,.,+,.,"ID=gene-b0002;Dbxref=EcoGene:EG10998,GeneID:94...",thrA
4,NC_000913.2,gene,2801,3733,.,+,.,"ID=gene-b0003;Dbxref=EcoGene:EG10999,GeneID:94...",thrB
6,NC_000913.2,gene,3734,5020,.,+,.,"ID=gene-b0004;Dbxref=EcoGene:EG11000,GeneID:94...",thrC
8,NC_000913.2,gene,5234,5530,.,+,.,"ID=gene-b0005;Dbxref=EcoGene:EG14384,GeneID:94...",yaaX


#### Drop more of the unneccesary columns....

In [50]:
df_gene.drop([5,7,8],axis=1,inplace=True)
df_gene.head()

,0,2,3,4,6,names
0,NC_000913.2,gene,190,255,+,thrL
2,NC_000913.2,gene,337,2799,+,thrA
4,NC_000913.2,gene,2801,3733,+,thrB
6,NC_000913.2,gene,3734,5020,+,thrC
8,NC_000913.2,gene,5234,5530,+,yaaX


#### BED files are specific in their column order, so we change the order of our columns so that it is in the correct format

In [51]:
cols=[0,3,4,2,'names',6]
df_gene=df_gene[cols]
df_gene.head()

,0,3,4,2,names,6
0,NC_000913.2,190,255,gene,thrL,+
2,NC_000913.2,337,2799,gene,thrA,+
4,NC_000913.2,2801,3733,gene,thrB,+
6,NC_000913.2,3734,5020,gene,thrC,+
8,NC_000913.2,5234,5530,gene,yaaX,+


#### BED files are also 0 indexed where GFF are 1 indexed, so we subtract one from the first coordinate
### WARNING: Only run this cell once or you'll end up subtracting more than one from the coordinate

In [52]:
df_gene[3]=df_gene[3]-1
df_gene.head()


,0,3,4,2,names,6
0,NC_000913.2,189,255,gene,thrL,+
2,NC_000913.2,336,2799,gene,thrA,+
4,NC_000913.2,2800,3733,gene,thrB,+
6,NC_000913.2,3733,5020,gene,thrC,+
8,NC_000913.2,5233,5530,gene,yaaX,+


### Save the file as a BED file with all gene intervals

In [53]:
df_gene.to_csv("NC_000913.2.bed",sep='\t',header=False,index=False)